# Using Codey to summarize SAP ABAP files

In [10]:
!pwd

/Users/pemelend/local_jupyter_lab/GCP-genAI-kickstart


In [1]:
# Authenticate (This should trigger oAuth to authenticate with end user's credentials using Appleconnect)
!gcloud auth application-default login

In [2]:
# Import libraries and initialize Vertex AI client
import vertexai
import google.auth
from vertexai.language_models import TextGenerationModel, CodeChatModel
from pathlib import Path
import pandas as pd
import pickle5 as pickle
import multiprocessing
cores_available = multiprocessing.cpu_count()

# Initialize Vertexai client
## To fill out by user
PROJECT_ID = 'multi-tenancy-dataproc' # @param {type:"string"}

if PROJECT_ID == "":
    PROJECT_ID = input("Enter your value: ")
LOCATION = 'us-central1'  # @param {type:"string"}

creds, _ = google.auth.default(quota_project_id=PROJECT_ID)
vertexai.init(project=PROJECT_ID, location=LOCATION, credentials=creds)

print(f'Using project ID: {PROJECT_ID}')

Using project ID: multi-tenancy-dataproc


In [13]:
# Files path Variables
cores_available = multiprocessing.cpu_count()

ROOT_PATH = "../sap_customer_programs"
MODEL = "codechat-bison-32k@002"
PROMPT = """You are a SAP ABAP expert. Provide a summary of the following SAP ABAP program and write output as bullets. \n"""

root_path = Path(ROOT_PATH)
program_list = list(root_path.rglob('*.[tT][xX][tT]'))
program_list = [[file.parent, file.name, str(file)] for file in program_list]
program_list_df = pd.DataFrame(program_list, columns =['File_Path', 'File_Name', 'Full_File_Name'])
program_list_df.head()

,File_Path,File_Name,Full_File_Name
0,../sap_customer_programs/Large/Z_IDOC_OUTPUT_C...,Z_IDOC_OUTPUT_CARR_TO_SAP_RU.TXT,../sap_customer_programs/Large/Z_IDOC_OUTPUT_C...
1,../sap_customer_programs/Large/ZRPV0404,ZRPV0404.txt,../sap_customer_programs/Large/ZRPV0404/ZRPV04...
2,../sap_customer_programs/Medium/ZFV45VFZY01 So...,ZFV45VFZY01 Source Code.TXT,../sap_customer_programs/Medium/ZFV45VFZY01 So...
3,../sap_customer_programs/Medium/ZRPF_FA_SUBLEDGER,ZRPF_FA_SUBLEDGER.TXT,../sap_customer_programs/Medium/ZRPF_FA_SUBLED...
4,../sap_customer_programs/Medium/ZRPF_FA_SUBLED...,ZRPF_FA_SUBLEDGER-checkpoint.TXT,../sap_customer_programs/Medium/ZRPF_FA_SUBLED...


In [12]:
def summarize_code_chat(full_file_name, model=MODEL, max_output_tokens= 1024, temperature = 0.3):

    parameters = {
            "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
            "max_output_tokens": max_output_tokens,  # Token limit determines the maximum amount of text output.
        }

    code_chat_model = CodeChatModel.from_pretrained(model)
    chat = code_chat_model.start_chat()
    
    # Using this line to grab only the first 2000 lines of each file and test summarizing all the files available
    abap_code = ''.join(Path(full_file_name).read_text().splitlines(True)[:2000])
    

    prompt = f"{PROMPT}{abap_code}"
    response = chat.send_message(
        prompt, **parameters
    )

    return response


from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI
# small_abap_pgm = Path("../sap_customer_programs/Small/zutv_upd_gtsblk_status/zutv_upd_gtsblk_status.txt").read_text()
small_abap_pgm = ''.join(Path("../sap_customer_programs/Large/ZRPV0404/ZRPV0404.txt").read_text().splitlines(True)[:2262])

chat = ChatVertexAI(
    model_name="codechat-bison-32k", max_output_tokens=1000, temperature=0.3
)

message = chat.invoke(f"{PROMPT}{small_abap_pgm}")
print(message.content)

In [15]:
import concurrent.futures
import urllib.request
import time

# This line filters only programs in the Small folder
full_file_names = program_list_df.Full_File_Name[program_list_df['Full_File_Name'].str.contains("Small")].to_list()

# full_file_names = program_list_df.Full_File_Name.to_list()

curr_time = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=cores_available) as executor:
    # Start the operations 
    future_to_summarize = {executor.submit(summarize_code_chat, full_file_name): full_file_name for full_file_name in full_file_names}

    for future in concurrent.futures.as_completed(future_to_summarize):
        program = future_to_summarize[future]
        try:
            data = future.result()
        except Exception as exc:
            print('\n\n %r generated an exception: %s' % (full_file_names, exc))
        else:
            print(f'\n ************************ New file ******************* \nProgram summarized is: {program} \nSummary is:\n\n{data.text[:]}')
print(f"\n Summarization completed in {time.time() - curr_time} seconds")


 ************************ New file ******************* 
Program summarized is: ../sap_customer_programs/Small/ZUTMZCPO/ZUTMZCPO.txt 
Summary is:

 **Summary of the SAP ABAP program:**

This SAP ABAP program, named "ZUTMZCPO", is designed to automate a series of tasks related to the processing of outbound deliveries and related documents. It focuses specifically on handling the following steps:

1. **Checking the Status:** The program begins by checking the status of a given delivery document (identified by its unique identifier "w_ibdeliv") to ensure that it's not already being processed or completed. If the status check passes, the program proceeds with subsequent steps.

2. **Data Retrieval:** The program retrieves relevant data from various SAP tables to gather information about the delivery document, purchase orders, and material documents associated with it. This data includes details such as material numbers, quantities, plants, storage locations, and more.

3. **Goods Receipt (

In [3]:
# Import libraries 
import dns.resolver 
  
# Finding A record 
result = dns.resolver.query('geeksforgeeks.org', 'A') 
  
# Printing record 
for val in result: 
    print('A Record : ', val.to_text())

A Record :  34.218.62.116


/var/folders/ny/mh9psmys4xsfh16x4b7c7jlw00ll23/T/ipykernel_8837/728754193.py:5: DeprecationWarning: please use dns.resolver.resolve() instead
  result = dns.resolver.query('geeksforgeeks.org', 'A')


In [1]:
import dnspython as dns
import dns.resolver

result = dns.resolver.query('tutorialspoint.com', 'A')
for ipval in result:
    print('IP', ipval.to_text())

ModuleNotFoundError: No module named 'dnspython'

In [2]:
from ping3 import ping, verbose_ping
verbose_ping('google.com')  # Returns delay in seconds.


ping 'google.com' ... 16ms
ping 'google.com' ... 16ms
ping 'google.com' ... 14ms
ping 'google.com' ... 17ms


# Langchain


### Using Langchain to summarize a single file with codechat-bison-32k

In [144]:
# Using Langchain to summarize a single file with codechat-bison-32k
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI
# small_abap_pgm = Path("../sap_customer_programs/Small/zutv_upd_gtsblk_status/zutv_upd_gtsblk_status.txt").read_text()

small_abap_pgm = ''.join(Path("../sap_customer_programs/Large/ZRPV0404/ZRPV0404.txt").read_text().splitlines(True)[:2306])

chat = ChatVertexAI(
    model_name="codechat-bison-32k", max_output_tokens=1000, temperature=0.3
)

message = chat.invoke(f"{PROMPT}{small_abap_pgm}")
print(message.content)

 The code you provided is an ABAP program that performs a series of tasks related to the processing and matching of sales orders and purchase orders in SAP. Here's a summary of what the code does:

**Initialization and Data Retrieval**:
- The program starts by initializing various internal tables and data structures.
- It then retrieves data from custom tables and SAP standard tables related to sales orders, purchase orders, and open sales orders.

**Matching Process**:
- The program attempts to match cancelled sales orders (with CIDs) with open sales orders based on various criteria such as material number, plant, storage location, and sales organization.
- It uses a series of loops and comparisons to find potential matches.

**Processing Matched Orders**:
- For each matched order, the program performs various actions:
  - It updates the custom table with the matched order information.
  - It closes the purchase requisition for the new order line item.
  - It performs stock movement f

### Langchain using map and reduce chain breaking down a bigger ABAP file

In [145]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAI as Lang_VertexAI

# This is a long document we need to split up in chunks.
abap_pgm_large = Path("../sap_customer_programs/Large/ZRPV0404/ZRPV0404.txt").read_text()
# abap_pgm_large = ''.join(Path("../sap_customer_programs/Large/ZRPV0404/ZRPV0404.txt").read_text().splitlines(True)[:2300])



# The splitter needed to be specifically after finding this separator
text_splitter = RecursiveCharacterTextSplitter(
    separators=["*&---------------------------------------------------------------------*"],
    chunk_size=80000,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False
)

pgm_chunks = text_splitter.create_documents([abap_pgm_large])



# Map
map_template = """You are a SAP ABAP expert. The following are sections of an SAP ABAP program, 
provide a summary and write output as bullets: {docs} """
map_prompt = PromptTemplate.from_template(map_template)

# Map chain with codechat-bison
chat_llm = ChatVertexAI(
    model_name="codechat-bison-32k", max_output_tokens=1024, temperature=0.3
)
map_chain = LLMChain(llm=chat, prompt=map_prompt)


# Reduce
reduce_template = """
Merge the following summaries of an SAP ABAP program: {docs} 
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)


# Reduce chain with text-bison
# text_llm = Lang_VertexAI(model_name="text-bison-32k@002", temperature=0.2)

reduce_chain = LLMChain(llm=chat_llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=1024,
)


# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

chain_run = map_reduce_chain.run(pgm_chunks)
print(chain_run)


 **Merged Summary**

The SAP ABAP program performs various tasks related to sales orders, purchase orders, and production orders. It includes functions for adjusting quantities, finding open records, retrieving data from related tables, calculating differences, creating and modifying purchase orders, reprocessing failed orders, creating deliveries, adding text to documents, updating audit tables, checking for matched orders, closing production orders, filtering matched orders, deleting production orders, getting customer groups, checking plant authorization, performing sales order authorization, and updating the status of records.

The program updates the ZT426 table with adjusted quantities, new records, and status changes. It creates or modifies purchase orders, creates deliveries, adds text to documents, updates audit tables with information about actions performed and errors encountered, filters and deletes matched orders based on criteria, performs authorization checks for plants 

In [146]:
len(pgm_chunks)

3